In [1]:
import random
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop

In [3]:
text_df = pd.read_csv("dataset/fake_or_real_news.csv")

In [11]:
text = list(text_df['text'].values)
joined_text = " ".join(text)

In [55]:
partial_text = joined_text[:1000000]

In [56]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [57]:
unique_tokens = np.unique(tokens)
unique_tokens_indexed = {token: idx for idx, token in enumerate(unique_tokens)}

In [58]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens)-n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [59]:
x = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)),dtype=bool)

In [60]:
for  i, words in enumerate(input_words):
    for j, word in enumerate(words):
        x[i, j, unique_tokens_indexed[word]] = 1
    y[i, unique_tokens_indexed[next_words[i]]] = 1

In [61]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [62]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(x,y, batch_size=128, epochs=50, shuffle=True)

Epoch 1/50
1318/1318 [==============================] - 649s 486ms/step - loss: 7.2300 - accuracy: 0.0608
Epoch 2/50
1318/1318 [==============================] - 649s 493ms/step - loss: 7.3832 - accuracy: 0.0863
Epoch 3/50
1318/1318 [==============================] - 659s 500ms/step - loss: 7.6546 - accuracy: 0.0968
Epoch 4/50
1318/1318 [==============================] - 653s 495ms/step - loss: 7.6136 - accuracy: 0.1054
Epoch 5/50
1318/1318 [==============================] - 7730s 6s/step - loss: 7.5373 - accuracy: 0.1105
Epoch 6/50
1318/1318 [==============================] - 645s 489ms/step - loss: 7.4420 - accuracy: 0.1153
Epoch 7/50
1318/1318 [==============================] - 642s 487ms/step - loss: 7.2880 - accuracy: 0.1236
Epoch 8/50
1318/1318 [==============================] - 636s 482ms/step - loss: 7.1100 - accuracy: 0.1336
Epoch 9/50
1318/1318 [==============================] - 718s 545ms/step - loss: 6.8342 - accuracy: 0.1483
Epoch 10/50
1318/1318 [=========================

In [63]:
model.save("better_model.h5")

In [64]:
model = load_model("better_model.h5")

In [47]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    x = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        x[0, i, unique_tokens_indexed[word]] = 1
    
    predictions = model.predict(x)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [65]:
possible = predict_next_word("he is the", 5)

1/1 [==============================] - 1s 904ms/step


In [66]:
print([unique_tokens[idx] for idx in possible])

['so', 'and', 'those', 'two', 'hate']


In [53]:
def generate_text(input_text, text_lenght, creativity= 3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_lenght):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence,creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current+=1
    return " ".join(word_sequence)

In [67]:
generate_text("he is the",100,5)

1/1 [==============================] - 0s 21ms/step


'he is the hate hate those hate june two 14 two the in 14 the in wait so hate 14 and so in 17 two and those so your 14 those your again those those two and again those 14 a 14 and and those your hate 14 so your should your 14 and again and hate your wait or those so those two so hate your 14 and 14 should 14 should those the those two those hate so hate your hate those again hate those or two two those and and your those so again those those new and democratic those'